In [1]:
import numpy as np
import pandas as pd

In [2]:
email_df = pd.read_csv('data-2024-07-16.csv', sep=';')

In [3]:
email_df.columns

Index(['Message ID', 'Received At', 'Processed At', 'Date', 'Trip Locator',
       'Customer', 'Trip Start', 'Trip End', 'Trip Start Date/time',
       'Trip End Date/time', 'Routing', 'Passengers', 'One Way', 'Source',
       'Aircraft', 'Source Suggested Price', 'Source Suggested Price.1',
       'Source Suggested Price.2', 'Source Suggested Price.3',
       'Source Suggested Price.4', 'Phone', 'Mobile', 'Email Subject'],
      dtype='object')

In [4]:
df = email_df[['Trip Locator', 'Customer', 'Trip Start Date/time', 'Trip End Date/time', 'Routing']]

In [5]:
df = df.dropna(subset=['Routing'], inplace = False)

In [6]:
df.head()

,Trip Locator,Customer,Trip Start Date/time,Trip End Date/time,Routing
0,DOKH8W,"Grant Casstevens (grant@dupageaerospace.com, 6...",07/16/2024 @ 03:30pm departure,NaN,BZN - SLC
1,Y62XE9,"Air Partner (nevin.jayawardena@airpartner.com,...",07/22/2024 @ 10:00am departure,07/28/2024 @ 09:00am departure,SDL - GEG - SDL
2,V6KM2F,Hunter Wrather (h.wrather@trilogyaviationgroup...,07/16/2024 @ 09:00am departure,07/19/2024 @ 05:00pm departure,FAR - MSP - MWCR - FAR
3,WAIWT1,"Wil Jordan (wjordan@luxury.aero, 631-676-7488)",07/24/2024 @ 10:00am departure,07/25/2024 @ 06:00pm departure,BCT - HOU - BCT
4,M4WZRZ,Nikita Volzhenin (n.volzhenin@primesk.kz),07/16/2024 @ 09:00am departure,NaN,SFO - MIA


In [7]:
df['Routing'] = df['Routing'].str.split(' - ')

In [8]:
df_filtered = df[df['Routing'].apply(lambda x: len(x) <= 3)].reset_index().drop(['index'], axis=1)

In [14]:
def route():
    origin_array = []
    destination_array = []
    return_array = []
    for i in df_filtered['Routing']:
        origin_array.append(i[0])
        destination_array.append(i[1])
        if len(i) > 2:
            return_array.append(i[2])
        else:
            return_array.append(None)
    origin_df = pd.DataFrame(origin_array, columns =['Origin'])
    destination_df = pd.DataFrame(destination_array, columns = ['Destination'])
    return_df = pd.DataFrame(return_array, columns = ['Return'])
    route_df = origin_df.join(destination_df)
    route_df = route_df.join(return_df)
    return route_df

route = route()

In [15]:
df_filtered.join(route)

,Trip Locator,Customer,Trip Start Date/time,Trip End Date/time,Routing,Origin,Destination,Return
0,DOKH8W,"Grant Casstevens (grant@dupageaerospace.com, 6...",07/16/2024 @ 03:30pm departure,NaN,"[BZN, SLC]",BZN,SLC,None
1,Y62XE9,"Air Partner (nevin.jayawardena@airpartner.com,...",07/22/2024 @ 10:00am departure,07/28/2024 @ 09:00am departure,"[SDL, GEG, SDL]",SDL,GEG,SDL
2,WAIWT1,"Wil Jordan (wjordan@luxury.aero, 631-676-7488)",07/24/2024 @ 10:00am departure,07/25/2024 @ 06:00pm departure,"[BCT, HOU, BCT]",BCT,HOU,BCT
3,M4WZRZ,Nikita Volzhenin (n.volzhenin@primesk.kz),07/16/2024 @ 09:00am departure,NaN,"[SFO, MIA]",SFO,MIA,None
4,UFIBD3,"Kobina Brew-Hammond (kobina@saharajets.com, 38...",07/19/2024 @ 07:00am departure,NaN,"[DFW, SFO]",DFW,SFO,None
5,GSLJUX,"Charter Team (charter@jet79.com, +1 (310) 888-...",08/25/2024 @ 10:00am departure,NaN,"[VNY, OPF]",VNY,OPF,None
6,TFGP8Y,"Jordan Brown (davido@flycfg.com, 888-634-7449)",08/03/2024 @ 10:00am departure,NaN,"[VNY, HNL]",VNY,HNL,None
7,DS11U6,Jonathan Scott (jonathan.scott@aircharterservi...,07/16/2024 @ 05:00pm departure,NaN,"[FXE, MMU]",FXE,MMU,None
8,RQSODM,"Thomas Nelson (tyler@trilogyaviationgroup.com,...",07/17/2024 @ 10:00am departure,NaN,"[FTW, MSP]",FTW,MSP,None
9,CBZITD,"Kaitlyn Wheatley (kwheatley@flyxo.com, +1 888 ...",07/20/2024 @ 09:00am departure,07/20/2024 @ 03:30pm departure,"[MVY, PWM, MVY]",MVY,PWM,MVY
